In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


from orbit.models import DLT
from orbit.diagnostics.backtest import BackTester, TimeSeriesSplitter
from orbit.diagnostics.plot import plot_bt_predictions
from orbit.diagnostics.metrics import smape, wmape
from orbit.utils.dataset import load_iclaims
from orbit.utils.plot import get_orbit_style

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
raw_data = load_iclaims()
data = raw_data.copy()

print(data.shape)
data.head(5)

(443, 7)


,week,claims,trend.unemploy,trend.filling,trend.job,sp500,vix
0,2010-01-03,13.386595,0.219882,-0.318452,0.117500,-0.417633,0.122654
1,2010-01-10,13.624218,0.219882,-0.194838,0.168794,-0.425480,0.110445
2,2010-01-17,13.398741,0.236143,-0.292477,0.117500,-0.465229,0.532339
3,2010-01-24,13.137549,0.203353,-0.194838,0.106918,-0.481751,0.428645
4,2010-01-31,13.196760,0.134360,-0.242466,0.074483,-0.488929,0.487404


In [4]:
data.dtypes

week              datetime64[ns]
claims                   float64
trend.unemploy           float64
trend.filling            float64
trend.job                float64
sp500                    float64
vix                      float64
dtype: object

In [5]:
n_splits = 6
forecast_len = 28 # future 4 week projection
incremental_len = 28

In [6]:
# instantiate a model
dlt = DLT(date_col='week',
             response_col='claims',
             regressor_col=['trend.unemploy', 'trend.filling', 'trend.job'],
             seasonality=52,
         estimator='stan-map')

In [7]:
bt = BackTester(model=dlt,
                df=data,
                incremental_len=incremental_len,
                n_splits=n_splits,
                forecast_len=forecast_len)

In [8]:
bt.fit_predict();

In [9]:
predicted_df = bt.get_predicted_df()
predicted_df.head()

,date,split_key,training_data,actual,prediction
0,2010-01-03,0,True,13.386595,13.386595
1,2010-01-10,0,True,13.624218,13.571237
2,2010-01-17,0,True,13.398741,13.429135
3,2010-01-24,0,True,13.137549,13.231236
4,2010-01-31,0,True,13.196760,13.124025


In [24]:
from orbit.diagnostics.plot import plot_bt_predictions2

Users can write it into multiple graphs and saves them into specific directory using `plot_bt_predictions2` and args `fig_dir`

In [25]:
plot_bt_predictions2(bt_pred_df=predicted_df, metrics=smape, figsize=(16, 8), is_visible=False, fig_dir='temp/',
                    markersize=80)

If user wants to export a `.gif` file, they can also use the args `export_gif=True`.

In [26]:
plot_bt_predictions2(bt_pred_df=predicted_df, metrics=smape, 
                     figsize=(16, 8), is_visible=False, fig_dir='temp/', export_gif=True,
                    markersize=80, imageio_args={'fps': 2.5})